In [131]:
import cnn_builder as cbuild
import config
import copy
import glob
import importlib
import lipiodol_methods as lm
import niftiutils.helper_fxns as hf
import niftiutils.liver_metrics as lmet
import niftiutils.transforms as tr
import niftiutils.registration as reg
import niftiutils.masks as masks
import numpy as np
import os
from os.path import *
import shutil
import SimpleITK as sitk
import sys
%matplotlib inline

In [94]:
importlib.reload(hf)
C = config.Config()

In [3]:
img_dir = "D:\\Lipiodol\\Images all"
seg_dir = "D:\\Lipiodol\\Images extracted and segmentations"
target_dir = "D:\\Lipiodol\\Data"

## Non-rigid reg

In [388]:
importlib.reload(hf)

<module 'niftiutils.helper_fxns' from 'c:\\users\\clinton\\documents\\niftiutils\\niftiutils\\helper_fxns.py'>

In [359]:
patient_id = "02"

mribl_art_path = join(target_dir, patient_id, "MRI-BL", "t1-art.nii.gz")
mribl_tumor_mask_path = join(target_dir, patient_id, "tumor bl.ids")
ct24_path = join(target_dir, patient_id, "nii_files", "ct24.nii.gz")
ct24_tumor_mask_path = glob.glob(join(target_dir, patient_id, "tumor 24h*.ids"))[0]

fix_tmp_path = join(target_dir, patient_id, "fix_tmp_path.nii")
mov_tmp_path = join(target_dir, patient_id, "mov_tmp_path.nii")
xform_path = join(target_dir, patient_id, "xform.txt")

img_fix_path = ct24_path
mask_fix_path = ct24_tumor_mask_path
img_mov_path = mribl_art_path
mask_mov_path = mribl_tumor_mask_path

In [360]:
ct_img, ct_dims = hf.dcm_load(join(target_dir, patient_id, "CT24h"), True, True)
hf.save_nii(ct_img, ct24_path, ct_dims)

In [446]:
img_f, img_f_dims = hf.nii_load(img_fix_path)
mask_f, mask_f_dims = masks.get_mask(mask_fix_path, img_f_dims, img_f.shape)
orig_shape = img_f.shape

img_m, img_m_dims = hf.nii_load(img_mov_path)
mask_m, mask_m_dims = masks.get_mask(mask_mov_path, img_m_dims, img_m.shape)

mask_f, crops_f = hf.crop_nonzero(mask_f)
img_f, _ = hf.crop_nonzero(img_f, crops_f)
img_f[mask_f == 0] = 0

mask_m, crops_m = hf.crop_nonzero(mask_m)
img_m, _ = hf.crop_nonzero(img_m, crops_m)
img_m[mask_m == 0] = 0

R = 1.5
mask_f_iso, _ = tr.rescale_img(mask_f, [R]*3, mask_f_dims)
img_f_iso, _ = tr.rescale_img(img_f, [R]*3, img_f_dims)
mask_m_iso, _ = tr.rescale_img(mask_m, [R]*3, mask_m_dims)
img_m_iso, _ = tr.rescale_img(img_m, [R]*3, img_m_dims)

mask_f_iso, pads_f = hf.zeropad(mask_f_iso, mask_m_iso.shape, ignore_neg=True)
mask_m_iso, pads_m = hf.zeropad(mask_m_iso, mask_f_iso.shape, ignore_neg=True)

hf.save_nii(mask_f_iso, fix_tmp_path, [R]*3)
hf.save_nii(mask_m_iso, mov_tmp_path, [R]*3)

crops_f = np.transpose(np.array(crops_f), (1,0))
crops_f[:,1] = orig_shape - (crops_f[:,1]+1)

In [447]:
reg.reg_masks(fix_tmp_path, mov_tmp_path, xform_path, verbose=False)

In [ ]:
# Interlude to produce enhancing mask

In [307]:
enh_mask_path = join(target_dir, patient_id, "enh tumor bl")
nec_mask_path = join(target_dir, patient_id, "nec tumor bl")
mribl_pre_path = join(target_dir, patient_id, "MRI-BL", "t1-pre.nii.gz")

In [308]:
art_img, dims = hf.nii_load(mribl_art_path)
pre_img, _ = hf.nii_load(mribl_pre_path)

In [309]:
mribl_liver_mask_path = join(target_dir, patient_id, "liver bl final.ids")
liver_mask, _ = masks.get_mask(mribl_liver_mask_path, dims, art_img.shape)
tumor_mask, _ = masks.get_mask(mribl_tumor_mask_path, dims, art_img.shape)

In [310]:
importlib.reload(lmet)
_ = lmet.segment_tumor(art_img, pre_img, liver_mask, tumor_mask, enh_mask_path, nec_mask_path, dims)

In [ ]:
# Transform using existing transform

In [444]:
def transform_mask(mask_path, img_path, xform_path, crops, pads, target_vox_dims, out_path=None,
                   R=1.5, tmp_path="D:\\temp.nii", tmp_path2="D:\\temp2.nii"):
    img, img_dims = hf.nii_load(img_path)
    mask, mask_dims = masks.get_mask(mask_path, img_dims, img.shape)
    crops_m, crops_f = crops
    pads_m, pads_f = pads
    
    mask, _ = crop_nonzero(mask, crops_m)
    mask, _ = tr.rescale_img(mask, [R]*3, mask_dims)
    mask = np.pad(mask, pads_m, 'constant')
    
    hf.save_nii(mask, tmp_path, [R]*3)
    reg.transform_img(tmp_path, xform_path, tmp_path2)
    xform_mask, _ = hf.nii_load(tmp_path2)
    
    xform_mask = xform_mask[pads_f[0][0]:xform_mask.shape[0]-pads_f[0][1],
                            pads_f[1][0]:xform_mask.shape[1]-pads_f[1][1],
                            pads_f[2][0]:xform_mask.shape[2]-pads_f[2][1]]
    xform_mask, _ = tr.rescale_img(xform_mask, target_vox_dims, [R]*3)
    
    xform_mask = xform_mask >= 128
    xform_mask = np.pad(xform_mask, crops_f, 'constant')
    
    if out_path is not None:
        masks.save_mask(xform_mask, out_path, target_vox_dims)
        
    return xform_mask

In [448]:
_ = transform_mask(enh_mask_path, img_mov_path, xform_path,
                     (crops_m, crops_f), (pads_m, pads_f), mask_f_dims, ct24_enh_mask_path)

### Compose with rotation

In [191]:
mribl_liver_mask_path = join(target_dir, patient_id, "liver bl final.ids")
ct24_liver_mask_path = glob.glob(join(target_dir, patient_id, "wholeliver_24hCT*.ids"))[0]

In [ ]:
mribl_ct24_xform, ct24_tumor, mribl_tumor, mribl_ct24_crops = full_reg_masks(ct24_tumor_mask_path, ct24_path, mribl_tumor_mask_path, mribl_art_path)

In [ ]:
# BL MRI to 24h CT
vec_field = lm.reg_masks(ct_tumor_mask_path, blmri_tumor_mask_path)

In [ ]:
mribl_enh_mask = reg.apply_field(vec_field)

In [ ]:
ct24_lip = lm.seg_lipiodol()

In [ ]:
reg.reg_sitk(ct24_lip, vec_field)

### MRI loading

In [114]:
mri_dcm_paths = [x for x in glob.glob(join(target_dir, '*', 'MRI-BL', '*20s*')) if '.nii' not in x]
mri_dcm_paths

In [ ]:
patient_id = "02"
dcm_paths = [x for x in glob.glob(join(target_dir, patient_id, "MRI-BL","*")) if x.find('.')==-1]
for dcm_path in dcm_paths:
    if not exists(splitext(dcm_path)[0] + ".nii.gz"):
        try:
            save_path = splitext(dcm_path)[0] + ".nii.gz"
            img, dims = hf.dcm_load(dcm_path, True, True)
            hf.save_nii(img, save_path, dims)
        except:
            print(dcm_path)

In [119]:
art_paths = [x for x in glob.glob(join(target_dir, patient_id, "MRI-BL","*.nii.gz")) if '20s' in x or 'Ph1-Ax' in x]
for path in art_paths:
    os.rename(path, join(dirname(path), "t1-art.nii.gz"))

In [ ]:
path_dict = {}
patient_id = '04'
for path in glob.glob(join(target_dir, patient_id, "MRI-BL","*","*")):
    header = hf.dcm_load_header(path)
    if len(header) > 0:
        series_descript = header[0][('0008', '103e')].value
        path_dict[path] = series_descript
        series_descript = series_descript.replace("/","-")
        os.rename(path, join(target_dir,patient_id, "MRI-BL", series_descript))

### Bias field corr

In [122]:
mri_raw_paths = glob.glob(join(target_dir, '*', 'MRI-BL', 't1-art.nii.gz'))
mri_bfc_paths = [join(dirname(x), "t1-art-bfc.nii.gz") for x in mri_raw_paths]

In [176]:
importlib.reload(masks)

<module 'niftiutils.masks' from 'c:\\users\\clinton\\documents\\niftiutils\\niftiutils\\masks.py'>

In [ ]:
glob.glob(join(target_dir, patient_id, ''

In [ ]:
masks.mask2nii()

In [128]:
inputImage = sitk.ReadImage(mri_raw_paths[0])
maskImage = sitk.OtsuThreshold( inputImage, 0, 1, 200 )

if len ( sys.argv ) > 3:
    inputImage = sitk.Shrink( inputImage, [ int(sys.argv[3]) ] * inputImage.GetDimension() )
    maskImage = sitk.Shrink( maskImage, [ int(sys.argv[3]) ] * inputImage.GetDimension() )

inputImage = sitk.Cast( inputImage, sitk.sitkFloat32 )

corrector = sitk.N4BiasFieldCorrectionImageFilter()

numberFilltingLevels = 4
#corrector.SetMaximumNumberOfIterations( [ int( sys.argv[5] ) ] *numberFilltingLevels  )

output = corrector.Execute( inputImage, maskImage )

sitk.WriteImage( output, mri_bfc_paths[0] )

### Masks

## Lipiodol seg

In [82]:
importlib.reload(lm)

<module 'niftiutils.masks' from 'c:\\users\\clinton\\documents\\niftiutils\\niftiutils\\masks.py'>

In [102]:
# Lipiodol 3 masks
info = {}
for seg in glob.glob(join(target_dir,"*","wholeliver_24hCT*.ids")):
    if dirname(seg) in info.keys():
        (ct_img, ct_dims, mask) = info[dirname(seg)]
    else:
        ct_img, ct_dims = hf.dcm_load(join(dirname(seg), "CT24h"), True, True)
        mask, _ = masks.get_mask(seg, ct_dims, ct_img.shape)
        
    lm.seg_lipiodol((mask > 0) * ct_img, dirname(seg), ct_dims)
    info[dirname(seg)] = (ct_img, ct_dims, mask)
    break

In [ ]:
# Lipiodol target vs non-target masks
info = {}
for seg in glob.glob(join(target_dir,"*","wholeliver_24hCT*.ids")):
    if dirname(seg) in info.keys():
        (ct_img, ct_dims, mask) = info[dirname(seg)]
    else:
        ct_img, ct_dims = hf.dcm_load(join(dirname(seg), "CT24h"), True, True)
        mask, _ = masks.get_mask(seg, ct_dims, ct_img.shape)
        
    lm.seg_target_lipiodol((mask > 0) * ct_img, dirname(seg), ct_dims)
    info[dirname(seg)] = (ct_img, ct_dims, mask)
    #break